Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [32]:
usersqty = ratings.groupby('userId').count()
usersqty100 = usersqty[usersqty['movieId']>=100].reset_index()
usersqty100.head()

,userId,movieId,rating,timestamp
0,4,204,204,204
1,5,100,100,100
2,8,116,116,116
3,15,1700,1700,1700
4,17,363,363,363


In [9]:
rmax = ratings.groupby('userId').max()[['timestamp']].reset_index()
rmax.head()

,userId,timestamp
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [10]:
rmin = ratings.groupby('userId').min()[['timestamp']].reset_index()
rmin.head()

,userId,timestamp
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [57]:
joined = usersqty100.merge(rmax, how='left', on='userId').merge(rmin, how='inner', on='userId')
joined = joined.drop(joined.columns[[1,2,3]],axis='columns')

joined.columns = ['userId','end','start']
joined['life_time'] = pd.to_datetime(joined.end, unit='s') - pd.to_datetime(joined.start, unit='s')
joined.head()

,userId,end,start,life_time
0,4,949982274,949778714,2 days 08:32:40
1,5,1163375145,1163373044,0 days 00:35:01
2,8,1154474527,1154389340,0 days 23:39:47
3,15,1469330735,997937239,5455 days 22:38:16
4,17,1127476640,1127468587,0 days 02:14:13


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов

In [85]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd
rzd.sum()

client_id        565
rzd_revenue    20941
dtype: int64

In [86]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto
auto.sum()

client_id         575
auto_revenue    63410
dtype: int64

In [87]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air
air.sum()

client_id      466
air_revenue    271
dtype: int64

In [61]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [88]:
clients = client_base['client_id']
df_base = clients.to_frame().reset_index()
merged = (
    df_base.merge(rzd, on='client_id', how='left')
     .merge(auto, on='client_id', how='left')
     .merge(air, on='client_id', how='left')
)
merged.loc['Total'] = merged.sum()
merged

,index,client_id,rzd_revenue,auto_revenue,air_revenue
0,0.0,111.0,1093.0,NaN,NaN
1,1.0,112.0,2810.0,NaN,NaN
2,2.0,113.0,10283.0,57483.0,NaN
3,3.0,114.0,5774.0,83.0,NaN
4,4.0,115.0,981.0,912.0,81.0
5,5.0,116.0,NaN,4834.0,4.0
6,6.0,117.0,NaN,98.0,13.0
7,7.0,118.0,NaN,NaN,173.0
Total,28.0,916.0,20941.0,63410.0,271.0


In [91]:
merged = (
    client_base.merge(rzd, on='client_id', how='left')
     .merge(auto, on='client_id', how='left')
     .merge(air, on='client_id', how='left')
)
merged.loc['Total'] = merged.sum()
merged

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
Total,916,Комсомольская 4Энтузиастов 8аЛевобережная 1аМи...,20941.0,63410.0,271.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Возможно, координаты можно использовать для анализа - в каких городах совершается меньше всего покупок, например, попробовать разобраться, почему это так. 
Данные по местоположению пользователей можно также использовать при планировании рекламной стратегии, она позволяет оценить популярность тех или иных товаров в разных регионах, потенциальные новые местоположения, можно оптимизировать логистику. Координаты можно использовать в визуализации на карте.

Если координаты одного и того же пользователя значительно удалены друг от друга, то для получения одной пары можно взять, например, наиболее часто повторяющиеся.
